In [1]:
using Pkg
Pkg.activate("../")
Pkg.instantiate()

using DataFrames, PlotlyJS, Plots, CSV

  Activating project at `~/Documents/IC2Project`


In [2]:
using MLJ, Combinatorics, MLJBase, LIBSVM, Clustering, LinearAlgebra

# Importando a Base

In [3]:
df = CSV.read("data/dataset_ic2023.csv", DataFrame)
df = rename(df, "Colesterol Total" => :ColesterolTotal)

df_train, df_test = MLJ.partition(df, 0.75, multi=true)

(599×4 DataFrame
 Row │ ColesterolTotal  Idade  Glicemia  Desfecho 
     │ Int64            Int64  Int64     Int64    
─────┼────────────────────────────────────────────
   1 │             254     38        93         1
   2 │             222     79        89        -1
   3 │             236     64       102        -1
   4 │             241     50       123        -1
   5 │             167     34        82        -1
   6 │             238     48        90         1
   7 │             233     35       129         1
   8 │             189     20       133        -1
  ⋮  │        ⋮           ⋮       ⋮         ⋮
 593 │             196     43       101        -1
 594 │             181     61       103        -1
 595 │             240     29        79        -1
 596 │             230     69        90        -1
 597 │             237     43        93        -1
 598 │             191     59        94        -1
 599 │             260     43       132         1
                                  

In [125]:
include("SubsetSelection.jl")
using .SubsetSelection

In [80]:
include("ModelEstimation.jl")
using .ModelEstimation

In [81]:
features = [:ColesterolTotal, :Idade, :Glicemia]
target = :Desfecho

X = Matrix(df_train[!, features])
y = df_train[:, target]

model = ModelEstimation.train_model(X, y)

Main.ModelEstimation.Model(Dict{Int64, Any}(4 => machine(LogisticClassifier(lambda = 2.220446049250313e-16, …), …), 2 => machine(LogisticClassifier(lambda = 2.220446049250313e-16, …), …), 3 => machine(LogisticClassifier(lambda = 2.220446049250313e-16, …), …), 1 => machine(LogisticClassifier(lambda = 2.220446049250313e-16, …), …)), Dict(4 => [2], 2 => [1, 2], 3 => [1], 1 => [2, 3]), Dict{Int64, Any}(4 => [181.14492753623188 46.58695652173913 141.68840579710144], 2 => [239.2312925170068 45.775510204081634 139.8639455782313], 3 => [179.68421052631578 41.80921052631579 89.49342105263158], 1 => [239.37654320987653 47.23456790123457 90.0]))

In [114]:
mach = machine((@load SVC pkg=LIBSVM verbosity=0)(kernel=LIBSVM.Kernel.Linear), X, y, scitype_check_level=0)
MLJBase.fit!(mach, verbosity=0)

trained Machine; caches model-specific representations of data
  model: SVC(kernel = Linear, …)
  args: 
    1:	Source @704 ⏎ AbstractMatrix{Count}
    2:	Source @147 ⏎ AbstractVector{Multiclass{2}}


In [116]:
ŷ = MLJBase.predict(mach, X)
accuracy(ŷ, y)

0.7595993322203672

In [117]:
ŷ = MLJBase.predict(mach, X_test)
accuracy(ŷ, y_test)

0.795

In [126]:
n_features = 3

input_log = SubsetSelection.SubsetSelectionInput(n_features, X, y, svm())

results = SubsetSelection.bestSubsetSelection(input_log)

([1, 2, 3], machine(SVC(kernel = RadialBasis, …), …))

In [ ]:
n_features = 3
features = [:ColesterolTotal, :Idade, :Glicemia]

X = Matrix(df_train[:, features])
G = X * X';
D = sqrt.(diag(G) .+ diag(G)' .- 2 .* G);

clust = hclust(D, linkage=:complete)
clusters = cutree(clust, k=4)
df_train[!,:Cluster] = clusters

In [ ]:
total = size(df_train)[1]
weights = Dict()
for cluster in unique(clusters)
    weights[cluster] = size(filter(row -> row[:Cluster] == cluster, df_train))[1] / total
end

In [121]:
# log = @load LogisticClassifier pkg=MLJLinearModels verbosity=0
svm = @load SVC pkg=LIBSVM verbosity=0
boost = @load XGBoostClassifier pkg=XGBoost verbosity=0

MLJXGBoostInterface.XGBoostClassifier

In [99]:
results = Dict()

target = :Desfecho

for cluster in unique(clusters)
    filtered_df = filter(row -> row[:Cluster] == cluster, df_train)
    X = Matrix(filtered_df[!, features])
    y = categorical(filtered_df[:, target])

    input_log = SubsetSelection.SubsetSelectionInput(n_features, X, y, log())

    results[cluster] = SubsetSelection.bestSubsetSelection(input_log)
end

UndefVarError: UndefVarError: clusters not defined

In [ ]:
avg_oof_acc = 0
for cluster in unique(clusters)
    avg_oof_acc += results[cluster][2][results[cluster][1]] * weights[cluster]
end
avg_oof_acc

In [ ]:
for cluster in unique(clusters)
    filtered_df = filter(row -> row[:Cluster] == cluster, df_train)
    X = Matrix(filtered_df[!, features])
    y = categorical(filtered_df[:, target])

    input_boost = SubsetSelection.SubsetSelectionInput(n_features, X, y, boost())

    results[cluster] = SubsetSelection.bestSubsetSelection(input_boost)
end

In [ ]:
X =  MLJ.table(Matrix(df_train[!, features]))
y = categorical(df_train[:, target])

mach = machine(svm(), X, y, scitype_check_level=0)

MLJBase.fit!(mach, verbosity=0)

ŷ_dist = MLJBase.predict(mach, X)
# ŷ = mode.(ŷ_dist)

acc = accuracy(ŷ_dist, y)

println("Accuracy: ", acc, " | Features: ", features)

In [ ]:
X_test = MLJ.table(Matrix(df_test[!, features]))
y_test = categorical(df_test[:, target])

ŷ_dist = MLJBase.predict(mach, X_test)
# ŷ = mode.(ŷ_dist)

acc = accuracy(ŷ_dist, y_test)
println("Accuracy: ", acc, " | Features: ", features)

In [ ]:
results

In [ ]:
avg_oof_acc = 0
for cluster in unique(clusters)
    avg_oof_acc += results[cluster][2][results[cluster][1]] * weights[cluster]
end
avg_oof_acc

In [ ]:
for cluster in unique(clusters)
    filtered_df = filter(row -> row[:Cluster] == cluster, df_train)
    X = Matrix(filtered_df[!, features])
    y = categorical(filtered_df[:, target])

    input_svm = SubsetSelection.SubsetSelectionInput(n_features, X, y, svm())

    results[cluster] = SubsetSelection.bestSubsetSelection(input_svm)
end

In [ ]:
avg_oof_acc = 0
for cluster in unique(clusters)
    avg_oof_acc += results[cluster][2][results[cluster][1]] * weights[cluster]
end
avg_oof_acc

In [ ]:
PlotlyJS.plot(PlotlyJS.scatter(
    x=df_train[!,:ColesterolTotal], y=df_train[!,:Idade], z=df_train[!,:Glicemia],
    mode="markers",
    marker=attr(
        size=6,
        color=df_train[!,:Desfecho],
        colorscale="Viridis",
        opacity=0.8
    ),
    type="scatter3d"
), Layout(margin=attr(l=0, r=0, b=0, t=0), showlegend=true))

In [ ]:
PlotlyJS.plot(df_train, x=:ColesterolTotal, y=:Idade, color=:Desfecho, mode="markers", type="scatter")

In [ ]:
PlotlyJS.plot(df_train, x=:ColesterolTotal, y=:Glicemia, color=:Desfecho, mode="markers", type="scatter")

In [ ]:
PlotlyJS.plot(df_train, x=:Idade, y=:Glicemia, color=:Desfecho, mode="markers", type="scatter")